# Exercise - Text Mining - Classification - SCIKIT-LEARN

We will predict the category of a product based on the description of the product.

**The unit of analysis is a product**

In [41]:
import pandas as pd
import numpy as np

In [42]:
products = pd.read_csv('products.csv')

In [43]:
products.head(5)

,Category,Description
0,Electronics,HP 680 Original Ink Advantage Cartridge (Black...
1,Clothing & Accessories,Bold N Elegant Navy Blue Thin Summer Pregnancy...
2,Books,The Travel Book: A Journey Through Every Count...
3,Electronics,Tiny Deal Compact 10x25 Mini Binoculars Telesc...
4,Clothing & Accessories,Nimble House 16Pcs/Set Unisex Women Men No Tie...


In [44]:
products.shape

(5000, 2)

## Assign the "target" variable



In [45]:
target = products['Category']

## Assign the "text" (input) variable

In [46]:
input_data = products[['Description']]

## Split the data

In [47]:
from sklearn.model_selection import train_test_split

train_set, test_set, train_y, test_y = train_test_split(input_data, target, test_size=0.3, random_state=42)

In [48]:
train_set.shape, train_y.shape

((3500, 1), (3500,))

In [49]:
test_set.shape, test_y.shape

((1500, 1), (1500,))

# Data Prep

In [50]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

from sklearn.preprocessing import FunctionTransformer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

## Sklearn: Text preparation

Create a pipeline to transform the text column and create a term by document matrix.

In [51]:
def new_col(df):
    #Create a copy so that we don't overwrite the existing dataframe
    df1 = df.copy()
    return np.array(df).ravel()
new_col(train_set)

array(['Dehman 100-Percent Silky Satin Hair Beauty Pillowcase, Black, King Size DEHMANSATIN Silk originated in INDIA and it is an honor to present this ancient oriental gift to you. The collective wisdom of the sericulture people of this land has been condensed DEHMAN has always adhered to its original heart from generation to generation. From raw satin silk material collection to processing and to finished products,each step is carefully selected. DEHMAN hopes that every customer will have a wonderful satin silk experience and enjoy the silky noble life. Why Choose DEHMAN Pillowcase ? Material：DEHMAN Satin Silk pillowcase is crafted by 19 momme pure mulberry silk. Both sides of the pillowcase is organic and natural satin silk.Light weight and easy to carry. Design: It is designed with hidden zipper closure.With Queen(20x30inches) and King(20x36inches) and Standard size(20x26inches) in various kinds colours. Quality：Superior durable plain color ,not easy to run after washing.Exquisite 

In [52]:
text_column = ['TEXT']

In [72]:
number_svd_components = 200

In [73]:
text_transformer = Pipeline(steps=[
                ('my_new_column', FunctionTransformer(new_col)),
                ('text', TfidfVectorizer(stop_words='english')),
                ('svd', TruncatedSVD(n_components=number_svd_components, n_iter=10))
            ])

In [74]:
preprocessor = ColumnTransformer([
                     ('text', text_transformer, text_column),
                    ],
        remainder='drop')

In [75]:
train_x = text_transformer.fit_transform(train_set)

train_x

array([[ 1.32127476e-01, -3.83815352e-02, -5.52127985e-02, ...,
        -3.18432061e-03, -5.09369194e-03,  3.34308688e-02],
       [ 1.19672838e-03,  1.09071177e-03, -9.41519044e-06, ...,
        -2.55964285e-02,  4.90886278e-02, -6.28797035e-03],
       [ 5.74701078e-02,  5.16487292e-02,  8.81057286e-03, ...,
         8.82875064e-03, -1.91797014e-02,  1.59020149e-02],
       ...,
       [ 1.59120292e-01, -7.66943792e-02, -1.60781830e-01, ...,
         3.98405316e-03,  1.69077619e-02,  2.09761786e-02],
       [ 8.31440381e-02, -4.33959857e-02, -3.44149656e-02, ...,
        -1.45956104e-02,  6.45951762e-03,  1.09312867e-02],
       [ 9.66538621e-02, -3.95970843e-02, -1.19857668e-01, ...,
         1.07796332e-02, -1.05505421e-02,  8.46845178e-05]])

In [76]:
test_x = text_transformer.transform(test_set)

test_x

array([[ 0.0197115 ,  0.0121762 ,  0.00564652, ...,  0.00514147,
        -0.01561196, -0.01598412],
       [ 0.08027585, -0.04060881,  0.02606995, ...,  0.04379239,
         0.02280426,  0.01112367],
       [ 0.17327444, -0.07025064, -0.0460695 , ..., -0.02952812,
         0.00274757, -0.01891412],
       ...,
       [ 0.08834409,  0.03663293,  0.00627167, ..., -0.02217824,
        -0.02299534,  0.01885421],
       [ 0.08826245, -0.03472008,  0.03563941, ..., -0.00914971,
         0.02224639, -0.01308575],
       [ 0.01982621, -0.00944311,  0.02909595, ...,  0.00937389,
        -0.0070566 ,  0.00682157]])

# Calculate the baseline

In [77]:
from sklearn.dummy import DummyClassifier

dummy_clf = DummyClassifier(strategy="most_frequent")

dummy_clf.fit(train_x, train_y)

DummyClassifier(strategy='most_frequent')

In [78]:
from sklearn.metrics import accuracy_score

In [79]:
dummy_train_pred = dummy_clf.predict(train_x)

baseline_train_acc = accuracy_score(train_y, dummy_train_pred)

print('Baseline Train Accuracy: {}' .format(baseline_train_acc))

Baseline Train Accuracy: 0.38142857142857145


In [80]:
dummy_test_pred = dummy_clf.predict(test_x)

baseline_test_acc = accuracy_score(test_y, dummy_test_pred)

print('Baseline Test Accuracy: {}' .format(baseline_test_acc))

Baseline Test Accuracy: 0.386


# Try one of the classifiers we have covered so far

In [81]:
from sklearn.ensemble import RandomForestClassifier 

In [82]:
rnd_clf = RandomForestClassifier(n_estimators=100, max_leaf_nodes=16, n_jobs=-1) 

rnd_clf.fit(train_x, train_y)

RandomForestClassifier(max_leaf_nodes=16, n_jobs=-1)

## Accuracy

In [83]:
train_y_pred = rnd_clf.predict(train_x)

train_acc = accuracy_score(train_y, train_y_pred)

print('Train acc: {}' .format(train_acc))

Train acc: 0.9068571428571428


In [84]:
test_y_pred = rnd_clf.predict(test_x)

test_acc = accuracy_score(test_y, test_y_pred)

print('Test acc: {}' .format(test_acc))

Test acc: 0.9013333333333333


## Generate the confusion matrix

In [85]:
from sklearn.metrics import confusion_matrix

confusion_matrix(test_y, test_y_pred)

array([[329,   5,   4,  27],
       [  1, 236,   2,  19],
       [ 11,   0, 235,  52],
       [  9,   9,   9, 552]], dtype=int64)

# Try another classifier we have covered so far

In [86]:
from sklearn.linear_model import SGDClassifier

sgd_clf = SGDClassifier(max_iter=100)


In [87]:
sgd_clf.fit(train_x, train_y)

SGDClassifier(max_iter=100)

## Accuracy

In [88]:
train_y_pred = sgd_clf.predict(train_x)

train_acc = accuracy_score(train_y, train_y_pred)

print('Train acc: {}' .format(train_acc))

Train acc: 0.9448571428571428


In [89]:
test_y_pred = sgd_clf.predict(test_x)

test_acc = accuracy_score(test_y, test_y_pred)

print('Test acc: {}' .format(test_acc))

Test acc: 0.9373333333333334


## Generate the confusion matrix

In [90]:
confusion_matrix(test_y, test_y_pred)

array([[335,   6,   5,  19],
       [  0, 253,   3,   2],
       [  9,   0, 272,  17],
       [ 10,  11,  12, 546]], dtype=int64)

# Check for the number of components

**Determine whether increasing/decreasing the number of components increases/decreases the two models' accuracies** Discuss your findings below.